<a href="https://colab.research.google.com/github/brayantole/Proyectos-Python/blob/main/Brayan_Tole_Taller_gestion_sql_y_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gestión Básica de Inventario con SQLite3
# Taller Gestión de Inventario con SQLite y Python

Brayan Camilo Tole Yara
Bogotá, Colombia



## Instalar librerías SQLite3

In [1]:
%%shell
sudo apt-get install sqlite3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 769 kB of archives.
After this operation, 1,874 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.5 [769 kB]
Fetched 769 kB in 1s (995 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package sqlite3.
(Reading dat

## Crear la base de datos y una tabla de inventario

In [2]:
import sqlite3
import random

# Crear la base de datos con SQlite
conn = sqlite3.connect('productos.db')
c = conn.cursor()

#Parte1
#PREGUNTA1
¿Qué acción realiza sqlite3.connect('productos.db') si el archivo no existe? ¿Y si ya existe?

Si el archivo productos.db no existe: El comando lo creará automáticamente en el directorio actual.

Si el archivo productos.db ya existe: El comando simplemente abrirá una conexión con esa base de datos existente, permitiéndote leer y escribir en ella.
#PREGUNTA2
¿Cuál es la función del objeto "cursor"? ¿Por qué no se ejecutan los comandos directamente sobre la conexión?

El objeto cursor actúa como un intermediario que permite ejecutar comandos SQL. Su función es recorrer los registros de la base de datos, ejecutar sentencias y administrar los resultados de una consulta.

No se ejecutan comandos directamente sobre el objeto de conexión (conn) porque la conexión maneja el enlace con el archivo de la base de datos, mientras que el cursor maneja la ejecución de los comandos. Esta separación permite, por ejemplo, tener múltiples cursores operando de forma independiente sobre la misma conexión, lo cual es útil para gestionar transacciones complejas.



In [3]:
# Crear una tabla llamada 'inventario'
c.execute('''
CREATE TABLE inventario (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    categoria TEXT,
    precio DECIMAL(10, 2),
    cantidad INTEGER,
    proveedor TEXT,
    fecha_ingreso DATE,
    ubicacion TEXT
)
''')


#PREGUNTA3
¿Qué dos funciones importantes cumple INTEGER PRIMARY KEY para la columna id?

Clave Primaria (PRIMARY KEY): Asegura que cada valor en la columna id sea único y no nulo. Funciona como el identificador exclusivo para cada fila de la tabla.

Autoincremento (INTEGER): En SQLite, cuando una columna se define como INTEGER PRIMARY KEY, se convierte en un alias de la rowid interna. Esto significa que la columna se autoincrementará automáticamente. Si insertas un nuevo producto sin especificar un id (usando None), SQLite le asignará el siguiente número entero disponible.

In [4]:
# Generar datos de ejemplo para la tabla
def generar_producto(id):
    nombres = ["Producto", "Artículo", "Objeto"]
    categorias = ["Electrónica", "Hogar", "Jardín", "Moda", "Deportes"]
    proveedores = ["Proveedor1", "Proveedor2", "Proveedor3"]
    ubicaciones = ["Almacén A", "Almacén B", "Almacén C"]

    nombre = random.choice(nombres) + str(id)
    categoria = random.choice(categorias)
    precio = round(random.uniform(10.0, 500.0), 2)
    cantidad = random.randint(1, 100)
    proveedor = random.choice(proveedores)
    fecha_ingreso = f"2023-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
    ubicacion = random.choice(ubicaciones)

    return (id, nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)

# Insertar 1000 productos en la tabla
productos = [generar_producto(i + 1) for i in range(10000)]
c.executemany('INSERT INTO inventario VALUES (?, ?, ?, ?, ?, ?, ?, ?)', productos)

# Guardar cambios y cerrar la conexión
conn.commit()
conn.close()

#PREGUNTA4
¿Cuál es la ventaja de usar c.executemany() en lugar de un bucle for con c.execute()?

La principal ventaja de executemany() es el rendimiento. Este método envía todos los datos a la base de datos en una sola operación, lo cual es mucho más eficiente que usar execute() dentro de un bucle.

Cada llamada a execute() implica una comunicación separada con la base de datos, lo que genera una sobrecarga significativa. Para miles de registros, executemany() reduce drásticamente el tiempo de ejecución.

#PREGUNTA5
¿Qué sucedería si se olvidara incluir la línea conn.commit()?

Si se olvidara incluir conn.commit(), los datos insertados no se guardarían permanentemente en el archivo productos.db. SQLite opera con transacciones; los cambios se mantienen en un estado temporal hasta que se confirman explícitamente con commit(). Al cerrar la conexión con conn.close() sin haber hecho commit, todos los cambios de esa transacción se perderían.

In [5]:
print("Base de datos creada y poblada con éxito.")

Base de datos creada y poblada con éxito.


## Consulta para mostrar todos los productos creados

In [6]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Consultar todos los productos
c.execute('SELECT * FROM inventario')
productos = c.fetchall()

# Imprimir los productos
for producto in productos:
    print(producto)

# Cerrar la conexión
conn.close()


(1, 'Producto1', 'Moda', 204.14, 63, 'Proveedor3', '2023-07-25', 'Almacén C')
(2, 'Objeto2', 'Deportes', 299.28, 75, 'Proveedor2', '2023-07-04', 'Almacén A')
(3, 'Producto3', 'Moda', 115.03, 67, 'Proveedor2', '2023-01-19', 'Almacén B')
(4, 'Objeto4', 'Jardín', 359.07, 25, 'Proveedor3', '2023-11-25', 'Almacén A')
(5, 'Artículo5', 'Moda', 463.08, 16, 'Proveedor1', '2023-05-20', 'Almacén C')
(6, 'Producto6', 'Hogar', 103.12, 17, 'Proveedor1', '2023-02-09', 'Almacén C')
(7, 'Objeto7', 'Hogar', 105.86, 62, 'Proveedor1', '2023-11-15', 'Almacén B')
(8, 'Artículo8', 'Moda', 464.74, 56, 'Proveedor2', '2023-03-20', 'Almacén A')
(9, 'Producto9', 'Electrónica', 408.36, 93, 'Proveedor1', '2023-02-25', 'Almacén A')
(10, 'Artículo10', 'Moda', 470.34, 68, 'Proveedor1', '2023-12-13', 'Almacén A')
(11, 'Objeto11', 'Hogar', 449.11, 82, 'Proveedor3', '2023-11-08', 'Almacén C')
(12, 'Producto12', 'Electrónica', 349.79, 44, 'Proveedor1', '2023-04-09', 'Almacén C')
(13, 'Artículo13', 'Deportes', 356.02, 48, 

#PARTE2
#PREGUNTA1
Escriba la consulta SQL necesaria para seleccionar únicamente las
columnas nombre, categoria y precio de todos los productos que pertenecen
al 'Proveedor1'.
# Escribir la consulta SQL
query_proveedor1 = """
SELECT
    nombre,
    categoria,
    precio
FROM
    inventario
WHERE
    proveedor = 'Proveedor1';
"""

## Realizar consulta y transferir a dataframe

In [7]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')

# Realizar la consulta y cargar los resultados en un DataFrame
query = "SELECT * FROM inventario WHERE precio > 450"
df_productos = pd.read_sql_query(query, conn)

# Mostrar el DataFrame
print(df_productos)

# Cerrar la conexión
conn.close()


        id         nombre categoria  precio  cantidad   proveedor  \
0        5      Artículo5      Moda  463.08        16  Proveedor1   
1        8      Artículo8      Moda  464.74        56  Proveedor2   
2       10     Artículo10      Moda  470.34        68  Proveedor1   
3       21     Producto21  Deportes  475.88        48  Proveedor1   
4       43     Producto43  Deportes  459.22        20  Proveedor2   
..     ...            ...       ...     ...       ...         ...   
979   9953     Objeto9953    Jardín  476.10        62  Proveedor2   
980   9971     Objeto9971      Moda  471.55        46  Proveedor3   
981   9983     Objeto9983      Moda  487.55        94  Proveedor1   
982   9992     Objeto9992      Moda  485.87         8  Proveedor3   
983  10000  Producto10000  Deportes  471.85         1  Proveedor3   

    fecha_ingreso  ubicacion  
0      2023-05-20  Almacén C  
1      2023-03-20  Almacén A  
2      2023-12-13  Almacén A  
3      2023-11-07  Almacén C  
4      2023-07-0

#PREGUNTA2
¿Qué ventajas ofrece pd.read_sql_query()?
El método pd.read_sql_query() de Pandas ofrece varias ventajas:

Simplicidad: Ejecuta la consulta y carga los resultados directamente en un DataFrame de Pandas en una sola línea de código.

Conveniencia: Elimina la necesidad de crear un cursor, ejecutar la consulta, usar fetchall() para obtener los datos y luego iterar sobre ellos para procesarlos.

Poder de Análisis: Una vez que los datos están en un DataFrame, puedes usar toda la potencia de la librería Pandas para análisis, manipulación, visualización y limpieza de datos de manera muy sencilla.

In [8]:
df_productos.head()

,id,nombre,categoria,precio,cantidad,proveedor,fecha_ingreso,ubicacion
0,5,Artículo5,Moda,463.08,16,Proveedor1,2023-05-20,Almacén C
1,8,Artículo8,Moda,464.74,56,Proveedor2,2023-03-20,Almacén A
2,10,Artículo10,Moda,470.34,68,Proveedor1,2023-12-13,Almacén A
3,21,Producto21,Deportes,475.88,48,Proveedor1,2023-11-07,Almacén C
4,43,Producto43,Deportes,459.22,20,Proveedor2,2023-07-02,Almacén A


## Nueva consulta

In [9]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')

# Realizar la consulta y cargar los resultados en un DataFrame
query = "SELECT * FROM inventario WHERE precio > 450 AND cantidad > 25"
df_productos = pd.read_sql_query(query, conn)

# Mostrar el DataFrame
print(df_productos)


# Cerrar la conexión
conn.close()

       id        nombre    categoria  precio  cantidad   proveedor  \
0       8     Artículo8         Moda  464.74        56  Proveedor2   
1      10    Artículo10         Moda  470.34        68  Proveedor1   
2      21    Producto21     Deportes  475.88        48  Proveedor1   
3      47    Producto47  Electrónica  498.20        34  Proveedor3   
4      59    Producto59     Deportes  480.45        74  Proveedor2   
..    ...           ...          ...     ...       ...         ...   
730  9921  Artículo9921        Hogar  486.84        88  Proveedor3   
731  9935  Producto9935       Jardín  477.95        79  Proveedor3   
732  9953    Objeto9953       Jardín  476.10        62  Proveedor2   
733  9971    Objeto9971         Moda  471.55        46  Proveedor3   
734  9983    Objeto9983         Moda  487.55        94  Proveedor1   

    fecha_ingreso  ubicacion  
0      2023-03-20  Almacén A  
1      2023-12-13  Almacén A  
2      2023-11-07  Almacén C  
3      2023-06-20  Almacén A  
4   

#PREGUNTA3
En la sección “Nueva consulta”, modifique la variable query para que, en lugar de mostrar productos con precio > 450 Y cantidad > 25, muestre aquellos que cumplan la condición precio > 480 O cantidad < 5.
SIMPLEMENTE TENDREMOS QUE CAMBIAR LA SIGUIENTE LINEA DE COMANDO
query = "SELECT * FROM productos WHERE precio > 480 OR cantidad < 5"

## Insertar nuevo producto

In [10]:
# Datos del nuevo producto
nuevo_producto = (
    None,  # SQLite genera un nuevo id automáticamente si la columna es autoincrement
    'NuevoProducto123',
    'Electrónica',
    249.99,
    20,
    'ProveedorX',
    '2024-04-16',
    'Almacén D'
)

# Conexión con la DB
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Insersión de nuevo producto
c.execute('INSERT INTO inventario VALUES (?, ?, ?, ?, ?, ?, ?, ?)', nuevo_producto)

# Guardar (commit) los cambios y cerrar conexión con la DB
conn.commit()
conn.close()

#PARTE 3
#PREGINTA1
¿Por qué es una buena práctica de seguridad usar ? como marcador de posición?

Usar ? (marcadores de posición) es la práctica recomendada para pasar valores a una consulta SQL porque previene ataques de inyección SQL. Cuando se usan estos marcadores, la librería de la base de datos se encarga de "sanitizar" los datos, es decir, se asegura de que los valores se traten únicamente como datos y no como parte del comando SQL. Esto evita que un atacante pueda insertar código SQL malicioso en los valores de entrada para manipular o dañar la base de datos.

## Insertar nuevo producto desde Excel

### Crear tabla excel de nuevos productos


In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(0)
num_prods = 5000

# Productos nuevos
nombres = ['Producto' + str(i) for i in range(num_prods)]
categorias = np.random.choice(['Electrónica', 'Hogar', 'Jardín', 'Moda', 'Deportes'], num_prods)
precios = np.round(np.random.uniform(10.0, 500.0, num_prods), 2)
cantidades = np.random.randint(1, 100, num_prods)
proveedores = np.random.choice(['Proveedor1', 'Proveedor2', 'Proveedor3'], num_prods)
fechas = [datetime.today() - timedelta(days=np.random.randint(0, 365)) for _ in range(num_prods)]
ubicaciones = np.random.choice(['Almacén A', 'Almacén B', 'Almacén C'], num_prods)

# DataFrame
df = pd.DataFrame({
    'Nombre': nombres,
    'Categoría': categorias,
    'Precio': precios,
    'Cantidad': cantidades,
    'Proveedor': proveedores,
    'Fecha de Ingreso': pd.to_datetime(fechas),
    'Ubicación': ubicaciones
})

print(df.head())

# Guardar el DataFrame en un archivo Excel
df.to_excel('nuevos_productos_inventario.xlsx', index=False)

      Nombre    Categoría  Precio  Cantidad   Proveedor  \
0  Producto0     Deportes  489.97        77  Proveedor1   
1  Producto1  Electrónica  347.86        78  Proveedor3   
2  Producto2         Moda   10.40        93  Proveedor2   
3  Producto3         Moda  402.25        51  Proveedor3   
4  Producto4         Moda  445.18        50  Proveedor1   

            Fecha de Ingreso  Ubicación  
0 2025-06-06 14:54:39.548867  Almacén B  
1 2025-03-11 14:54:39.548918  Almacén C  
2 2024-12-17 14:54:39.548935  Almacén B  
3 2025-08-14 14:54:39.548948  Almacén A  
4 2024-11-30 14:54:39.549012  Almacén A  


### Agregar datos de tabla excel al inventario

In [12]:
import pandas as pd

# Leer archivo Excel
ruta_archivo = '/content/nuevos_productos_inventario.xlsx'
df = pd.read_excel(ruta_archivo)

# Verificación de formatos (tipo de dato)
# Precio a float
if df['Precio'].dtype != 'float':
    df['Precio'] = df['Precio'].astype(float)

# Fecha de Ingreso a datetime
if df['Fecha de Ingreso'].dtype != 'datetime64[ns]':
    df['Fecha de Ingreso'] = pd.to_datetime(df['Fecha de Ingreso'], errors='coerce')

# Verificar y corregir fechas no válidas, usando la fecha de hoy
df['Fecha de Ingreso'].fillna(pd.Timestamp.today(), inplace=True)

df.head()

/tmp/ipython-input-1041989223.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Fecha de Ingreso'].fillna(pd.Timestamp.today(), inplace=True)


,Nombre,Categoría,Precio,Cantidad,Proveedor,Fecha de Ingreso,Ubicación
0,Producto0,Deportes,489.97,77,Proveedor1,2025-06-06 14:54:39.549,Almacén B
1,Producto1,Electrónica,347.86,78,Proveedor3,2025-03-11 14:54:39.549,Almacén C
2,Producto2,Moda,10.40,93,Proveedor2,2024-12-17 14:54:39.549,Almacén B
3,Producto3,Moda,402.25,51,Proveedor3,2025-08-14 14:54:39.549,Almacén A
4,Producto4,Moda,445.18,50,Proveedor1,2024-11-30 14:54:39.549,Almacén A


#PREGUNTA2
¿Por qué es fundamental validar los tipos de dato antes de la inserción?

Es fundamental validar y convertir los tipos de datos antes de insertarlos porque las bases de datos tienen esquemas estrictos. Cada columna en la tabla inventario fue definida con un tipo de dato específico (TEXT, DECIMAL, INTEGER, DATE).

Si intentamos insertar un dato con un formato incorrecto (por ejemplo, un texto en una columna de precios o una fecha mal formateada), la base de datos rechazará la operación y generará un error. Realizar la validación en el código de Python permite manejar estos problemas de antemano (por ejemplo, convirtiendo los datos al formato correcto o estableciendo un valor por defecto), garantizando la integridad de los datos y evitando que el programa se detenga por errores inesperados.

In [16]:
import sqlite3

# Conexión con la DB
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Insersión de datos en la DB
for index, row in df.iterrows():
    c.execute("""
        INSERT INTO inventario (id, nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", (None, row['Nombre'], row['Categoría'], row['Precio'], row['Cantidad'], row['Proveedor'], str(row['Fecha de Ingreso']), row['Ubicación']))
###CORRECION None,.....###

# # Insersión de datos en la DB
# for index, row in df.iterrows():
#     c.executemany("""
#         INSERT INTO inventario (nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)
#         VALUES(?, ?, ?, ?, ?, ?,)""",
#         [row['Nombre'], row['Categoría'], row['Precio'], row['Cantidad'], row['Proveedor'], row['Fecha de Ingreso'], row['Ubicación']])

# Guardar cambios y cerrar conexión con DB
conn.commit()
conn.close()

print("¡Datos importados exitosamente!")


¡Datos importados exitosamente!


#PREGUNTA3
¡Error en el código!

A:
El error se encuentra en la tupla de valores que se pasa al método c.execute():

(row[], row['Nombre'], row['Categoría'], ...)

El primer elemento, row[], es una sintaxis inválida en Python. La intención era pasar un valor para la columna id, pero no se especificó ninguna clave dentro de los corchetes. Además, como el id es autoincremental, no deberíamos intentar asignarle un valor del DataFrame directamente

B: Para corregirlo, debemos reemplazar row[] por None, para indicarle a SQLite que genere automáticamente el valor del id. El bloque de código corregido se vería así:

(None, row['Nombre'], row['Categoría'], row['Precio'], row['Cantidad'], row['Proveedor'], str(row['Fecha de Ingreso']), row['Ubicación']))

#PARTE 4
#PREGUNTA1
optimizacion


In [22]:
import sqlite3
import pandas as pd

# Leer el archivo Excel y asegurar los tipos de datos (como en la sección anterior)
ruta_archivo = '/content/nuevos_productos_inventario.xlsx'
df = pd.read_excel(ruta_archivo)
df['Precio'] = df['Precio'].astype(float)
df['Fecha de Ingreso'] = pd.to_datetime(df['Fecha de Ingreso'], errors='coerce').fillna(pd.Timestamp.today())
df['Fecha de Ingreso'] = df['Fecha de Ingreso'].dt.strftime('%Y-%m-%d %H:%M:%S') # Convertir fecha a texto para SQLite

# Conexión con la DB
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Preparar los datos como una lista de tuplas
# Se agrega 'None' al inicio de cada tupla para el ID autoincremental
datos_para_insertar = [(None,) + tuple(row) for row in df.to_numpy()]

# Usar executemany para una inserción masiva y eficiente
c.executemany("""
    INSERT INTO inventario (id, nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", datos_para_insertar)

# Guardar cambios y cerrar conexión
conn.commit()
conn.close()

print(f"¡{len(datos_para_insertar)} nuevos registros importados exitosamente usando executemany!")


¡5000 nuevos registros importados exitosamente usando executemany!


#PREGUNTA2
agregacion

In [20]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')

# Consulta SQL para calcular el valor total por categoría y ordenar los resultados
query_valor_total = """
SELECT
    categoria,
    SUM(precio * cantidad) AS valor_total_inventario
FROM
    inventario
GROUP BY
    categoria
ORDER BY
    valor_total_inventario DESC;
"""

# Ejecutar la consulta y cargar los resultados en un DataFrame
df_valor_total = pd.read_sql_query(query_valor_total, conn)

# Formatear la columna de valor para que se vea como moneda (opcional, para mejor visualización)
df_valor_total['valor_total_inventario'] = df_valor_total['valor_total_inventario'].apply(lambda x: f"${x:,.2f}")


# Mostrar el DataFrame con los resultados
print("Valor total del inventario por categoría:")
print(df_valor_total)

# Cerrar la conexión
conn.close()

Valor total del inventario por categoría:
     categoria valor_total_inventario
0        Hogar         $39,898,681.86
1         Moda         $39,838,542.90
2     Deportes         $38,940,629.59
3  Electrónica         $37,209,759.00
4       Jardín         $37,147,009.62
